In [20]:
import numpy as np
from sympy import Matrix, zoo

class SimplexSolver:
    def __init__(self, filename):
        self.data = SimplexData(filename)
        self.epsilon = 0.5  # Equivalent to pow(2, -1)

    def solve(self):
        '''
        Driver function which calls the subroutines according to the Simplex Algorithm.
        '''
        try:
            if not self.data.is_feasible(self.data.start_point):
                raise ValueError(f'Initial point {self.data.start_point} is not feasible.')

            solved = False

            while not solved:
                # Step 2: Move towards the first vertex
                while not self.data.is_vertex(self.data.start_point):
                    self.data.start_point = self.move(self.data.start_point)

                # Test for degeneracy of the vertex
                if self.data.is_degenerate(self.data.start_point):
                    self.data.vector_b = self.remove_degeneracy(self.data.original_b.copy())
                    self.epsilon *= 2
                    continue

                # Print the value of the Cost function at this first vertex
                print(f'Vertex {self.data.start_point} --> Cost: {np.dot(self.data.start_point, self.data.vector_c)}')

                # Step 3: March through every vertex and check for optimum along with degeneracy
                solution = True
                degenerate = False
                while solution and (not self.data.is_optimum(self.data.start_point)):
                    self.data.start_point, solution = self.optimize(self.data.start_point)
                    if not solution:
                        raise ValueError('No solution found during optimization.')
                    elif self.data.is_degenerate(self.data.start_point):
                        self.data.vector_b = self.remove_degeneracy(self.data.original_b.copy())
                        degenerate = True
                        self.epsilon *= 2
                        break
                    else:
                        print(f'Vertex {self.data.start_point} --> Cost: {np.dot(self.data.start_point, self.data.vector_c)}')

                if not degenerate:
                    solved = True

            # Convert the solution back to the original solution, if it exists
            if solution:
                X, y = [], []
                for i in range(len(self.data.matrix_A)):
                    if np.isclose(np.dot(self.data.matrix_A[i], self.data.start_point), self.data.vector_b[i]):
                        X.append(self.data.matrix_A[i])
                        y.append(self.data.original_b[i])
                X = np.array(X)
                y = np.array(y)
                try:
                    z_solution = np.linalg.solve(X, y)
                except np.linalg.LinAlgError:
                    raise np.linalg.LinAlgError('Final system is singular; cannot find unique solution.')
                print(f'Original Solution: Vertex {z_solution} --> Cost: {np.dot(z_solution, self.data.vector_c)}')

        except Exception as e:
            print(f'Error: {e}')

    def move(self, start_point):
        '''
        Moves the feasible point towards a vertex of the feasible region by one step size.
        '''
        try:
            m, n = self.data.matrix_A.shape
            A_tight, _, A_untight, b_untight = self.data.split_rows(start_point)
            A_tight = A_tight.reshape((A_tight.shape[0], n))

            # Compute the direction to move the feasible point
            nullspace = np.array(Matrix(A_tight).nullspace())
            if nullspace.size == 0:
                raise ValueError('Nullspace of A_tight is empty; cannot move towards vertex.')
            scale = np.random.randint(1, 4, nullspace.shape[0]).reshape(nullspace.shape[0], 1, 1)
            direction = np.array(np.sum(nullspace * scale, axis=0)).astype(float)
            u = direction / np.linalg.norm(direction)

            # Compute the step size 'alpha'
            temp = np.dot(A_untight, u)
            temp[np.isclose(temp, 0)] = 1e-32  # Avoid division by zero
            alpha_values = (b_untight - np.dot(A_untight, start_point)) / temp.T
            alpha_values = alpha_values[0]
            alpha_values = np.array([a for a in alpha_values if not ((a > 1e6) or (-a) > 1e6)])
            if alpha_values.size == 0:
                raise ValueError('No valid alpha found; cannot move towards vertex.')
            alpha = alpha_values[np.argmin(np.abs(alpha_values.copy()))]

            # Update the feasible point by one step size
            u = u.T
            new_start_point = start_point + alpha * u
            new_start_point = new_start_point[0]

            return new_start_point
        except Exception as e:
            raise RuntimeError(f'Error in moving towards vertex: {e}')

    def optimize(self, start_point):
        '''
        Optimizes the cost function by moving to the next optimum vertex.
        '''
        try:
            A_tight, _, A_untight, b_untight = self.data.split_rows(start_point)

            # Compute the direction to move in order to reach a more optimum vertex
            A_tight_inverse = np.linalg.inv(A_tight)
            A_tight_inverse = np.transpose(A_tight_inverse)
            u = None
            for x in A_tight_inverse:
                if np.dot(-x, self.data.vector_c.T) > 0:
                    u = x.T
                    break
            if u is None:
                raise ValueError('No direction found to optimize the cost function.')

            # Compute the step size 'alpha'
            temp = np.dot(A_untight, u)
            temp[np.isclose(temp, 0)] = 1e-16  # Avoid division by zero
            alpha_values = (np.dot(A_untight, start_point) - b_untight) / temp
            alpha_values = [x for x in alpha_values if x != zoo]
            if all([a < 0 for a in alpha_values]):
                return start_point, False
            alpha = min([x for x in alpha_values if x >= 0])

            # Update the vertex
            new_start_point = start_point - alpha * u

            return new_start_point, True
        except np.linalg.LinAlgError:
            raise np.linalg.LinAlgError('Matrix A_tight is singular; cannot compute inverse.')
        except Exception as e:
            raise RuntimeError(f'Error in optimization step: {e}')

    def remove_degeneracy(self, vector_b):
        '''
        Removes degeneracy from the feasible region by adding powers of a small positive number to vector b.
        '''
        add = self.epsilon
        for i in range(len(vector_b)):
            vector_b[i] += add
            add *= self.epsilon
        return vector_b

class SimplexData:
    def __init__(self, filename):
        self.matrix_A, self.vector_b, self.vector_c, self.start_point = self.read_csv(filename)
        self.original_b = self.vector_b.copy()

    def read_csv(self, filename):
        '''
        Reads the input matrix A, vectors b and c, and initial point z from a CSV file.
        '''
        try:
            with open(filename, 'r') as input_file:
                # Read the first line to extract vector z
                line = input_file.readline()
                if not line:
                    raise ValueError('Input file is empty or does not contain initial point z.')
                start_point = list(map(float, line.strip().split(',')))
                start_point = np.asarray(start_point[:-1], dtype=float)

                # Read the second line to extract vector c
                line = input_file.readline()
                if not line:
                    raise ValueError('Input file does not contain cost vector c.')
                vector_c = list(map(float, line.strip().split(',')))
                vector_c = np.asarray(vector_c[:-1], dtype=float)

                # Read all the remaining lines to extract matrix A and vector b
                matrix_A, vector_b = [], []
                for line in input_file:
                    l = list(map(float, line.strip().split(',')))
                    matrix_A.append(l[:-1])
                    vector_b.append(l[-1])
                if not matrix_A or not vector_b:
                    raise ValueError('Input file does not contain matrix A and vector b.')

                matrix_A = np.asarray(matrix_A, dtype=float)
                vector_b = np.asarray(vector_b, dtype=float)

            return matrix_A, vector_b, vector_c, start_point
        except FileNotFoundError:
            raise FileNotFoundError(f'File "{filename}" not found.')
        except Exception as e:
            raise ValueError(f'Error reading file "{filename}": {e}')

    def is_feasible(self, start_point):
        return np.all(np.matmul(self.matrix_A, start_point) <= self.vector_b)

    def split_rows(self, start_point):
        A_tight, A_untight, b_tight, b_untight = [], [], [], []
        Az = np.dot(self.matrix_A, start_point)
        for i in range(len(Az)):
            if np.isclose(np.dot(self.matrix_A[i], start_point), self.vector_b[i]):
                A_tight.append(self.matrix_A[i])
                b_tight.append(self.vector_b[i])
            else:
                A_untight.append(self.matrix_A[i])
                b_untight.append(self.vector_b[i])
        return np.array(A_tight), np.array(b_tight), np.array(A_untight), np.array(b_untight)

    def is_vertex(self, start_point):
        A_tight, _, _, _ = self.split_rows(start_point)
        rank_A = np.linalg.matrix_rank(self.matrix_A)
        num_tight_rows = A_tight.shape[0]
        return rank_A <= num_tight_rows

    def is_degenerate(self, start_point):
        A_tight, _, _, _ = self.split_rows(start_point)
        return np.linalg.matrix_rank(self.matrix_A) < A_tight.shape[0]

    def is_optimum(self, start_point):
        try:
            A_tight, _, _, _ = self.split_rows(start_point)
            beta = np.dot(np.linalg.inv(A_tight.T), self.vector_c.T)
            return np.all(beta >= 0)
        except np.linalg.LinAlgError:
            raise np.linalg.LinAlgError('Matrix A_tight.T is singular; cannot compute beta.')
        except Exception as e:
            raise RuntimeError(f'Error in testing for optimum: {e}')


if __name__ == '__main__':
    # Replace the filename with the path to your input CSV file
    filename = r'neha.csv'
    solver = SimplexSolver(filename)
    solver.solve()


Vertex [-0.1875  4.25  ] --> Cost: -16.4375
Vertex [0.375 3.125] --> Cost: -13.625
Vertex [ 3.625 -0.125] --> Cost: -10.375
Original Solution: Vertex [ 4. -0.] --> Cost: -12.0
